In [23]:
from datasets import load_dataset

# Load the English dataset from the latest dump
ds_wiki = load_dataset("omarkamali/wikipedia-monthly", "20250703.tr", split="train")
ds_wiki

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset({
    features: ['id', 'url', 'title', 'raw_mediawiki', 'text'],
    num_rows: 641443
})

In [26]:
# first 100 to text with title
text = ""
for i in range(100):
    text += ds_wiki[i]["title"] + "\n" + ds_wiki[i]["text"] + "\n"

while "\n " in text:
    text = text.replace("\n ", "\n")

while "\n\n" in text:
    text = text.replace("\n\n", "\n")


with open("data.txt", "w", encoding="utf-8") as f:
    f.write(text)

print(len(text))

1394660


In [6]:
%pip install turkish-tokenizer -U


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m-it")
tokenizer.name_or_path

'google/gemma-3-270m-it'

In [2]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m-it")


In [11]:
from gemma_model import GemmaForCausalLM, get_config_for_270m

config_270m = get_config_for_270m("float32")

gemma_model = GemmaForCausalLM(config_270m, tokenizer)
gemma_model.load_weights_from_hf(model.model.state_dict())
# gemma_model.from_pretrained("gemma-3-270m-hf-it")


HF model vocab size: 262144, Current model vocab size: 262144
Missing keys: ['local_freqs_cis', 'global_freqs_cis']
Successfully loaded 236 weights


In [13]:
gemma_model.state_dict().keys()

odict_keys(['local_freqs_cis', 'global_freqs_cis', 'model.embedder.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.query_norm.weight', 'model.layers.0.self_attn.key_norm.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.pre_feedforward_layernorm.weight', 'model.layers.0.post_feedforward_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.query_norm.weight', 'model.layers.1.self_attn.key_norm.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.dow

In [6]:
prompt = "Merhaba, nasılsın?"

In [4]:
import torch
# Automatic device detection
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [14]:
gemma_model.generate(prompt)


' aí emotionκτη na فلسطینی na فلسطینی na فلسطینی na فلسطینی na فلسطینی na na na je فلسطینی na je فلسطینی na فلسطینی na je فلسطینی na الفلسطيني и فلسطینی je الفلسطيني je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطین, je فلسطینی je فلسطین je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطینی je فلسطین je فلسطین je فلسطینی je فلسطین je فلسطینی je فلسطینی je فلسطین je فلسطینی je فلسطین je فلسطین je فلسطینی je فلسطین je فلسطین Je فلسطینی je فلسطینی je فلسطین je'

In [4]:
gemma_model.save_pretrained("gemma-3-270m-hf-it")

Model saved to gemma-3-270m-hf-it


In [2]:
from turkish_tokenizer import TurkishTokenizer

# Tokenizer'ı başlat
tr_tokenizer = TurkishTokenizer()

# Metin tokenizasyonu
text = """
Ali Ata Bak▁ aliler ahmetler selmanlar da bizde onlar da testte kitapta kitabını okudum bu işe
 
 burnunu sokma
Burun buruna bir kaza\toldu
 """
tokens = tr_tokenizer.encode(text)
print("Token IDs:", tokens)

# Token'ları metne geri çevir
decoded_text = tr_tokenizer.decode(tokens)
print("Decoded:", decoded_text)

Token IDs: [3, 0, 2697, 2, 0, 2212, 2, 0, 2794, 1, 2, 18194, 20043, 2, 6766, 20000, 2, 17321, 20000, 2, 20024, 2, 2595, 20024, 2, 2627, 2, 20024, 2, 3045, 20024, 2, 227, 20024, 2, 227, 15247, 2, 2656, 10572, 2, 2503, 2, 2599, 20038, 3, 3, 2, 165, 20021, 20035, 2, 20064, 4373, 20002, 3, 0, 165, 2, 165, 20037, 2, 2501, 2, 3303, 4, 2502, 20026, 3, 2]
Decoded: 
Ali Ata Bak▁u▁ aliler ahmetler selmanlar da bizde onlar da testte kitapta kitabını okudum bu işe

 burnunu sokma
Burun burna bir kaza	oldu
 


In [12]:
from text_dataset import create_dataset, create_dataloader

with open("data.txt", "r", encoding="utf-8") as f:
    text = f.read()

seq_len = 512
stride = 1
batch_size = 12
shuffle = False

train_dataset = create_dataset(text[:10000], tr_tokenizer, seq_len, stride, device)
# dataset = create_or_load_dataset(text[:50000], tr_tokenizer, seq_len, stride, device, cache_path="dataset/wiki1")
train_dataloader = create_dataloader(train_dataset, batch_size, shuffle)



In [13]:
valid_dataset = create_dataset(text[10000:12000], tr_tokenizer, seq_len, stride, device)
valid_dataloader = create_dataloader(valid_dataset, batch_size, shuffle)

In [14]:
train_dataset.get_sequence_info(), valid_dataset.get_sequence_info()

({'seq_len': 512,
  'stride': 1,
  'num_sequences': 3875,
  'tokenizer_name': 'unknown'},
 {'seq_len': 512,
  'stride': 1,
  'num_sequences': 503,
  'tokenizer_name': 'unknown'})

In [5]:
from gemma_model import GemmaForCausalLM, get_config_for_270m_tr_tokenizer


config_270m = get_config_for_270m_tr_tokenizer("float32")

gemma_model = GemmaForCausalLM(config_270m, tr_tokenizer, device)
# gemma_model.load_weights_from_hf(model.model.state_dict())
gemma_model.from_pretrained("gemma-3-270m-tr-tokenizer-it")


Model loaded from gemma-3-270m-tr-tokenizer-it


In [6]:
gemma_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embedder): Embedding(32768, 640)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (query_norm): RMSNorm()
          (key_norm): RMSNorm()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (pre_feedforward_layernorm): RMSNorm()
        (post_feedforward_layernorm): RMSNorm()
 

In [16]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from model_trainer import ModelTrainer

optimizer = optim.AdamW(gemma_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# 3. Instantiate and Run the Trainer
trainer = ModelTrainer(
    model=gemma_model,
    train_loader=train_dataloader,
    val_loader=valid_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    device=device,
    epochs=10,
    patience=3
)

trainer.train()

print("\nTrainer has finished. The `trainer.model` now holds the best weights.")

Starting training on device: mps


Epoch 01/10 | Train Loss: 0.5769 | Val Loss: 13.9190 | LR: 0.001000
Validation loss decreased (inf --> 13.9190). Saving model...


KeyboardInterrupt: 

In [ ]:
gemma_model.generate(prompt)

TypeError: index_copy_() received an invalid combination of arguments - got (int, NoneType, Tensor), but expected one of:
 * (int dim, Tensor index, Tensor source)
      didn't match because some of the arguments have invalid types: (int, !NoneType!, Tensor)
 * (name dim, Tensor index, Tensor source)
      didn't match because some of the arguments have invalid types: (!int!, !NoneType!, Tensor)


In [4]:

# Generate text (no device parameter needed)
response = gemma_model.generate(
    prompts="Hello, how are you?",
    output_len=50,
    temperature=0.7
)

In [6]:
response

'ek_temp_20212ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244ek_temp_20244'

In [5]:
gemma_model.generate(prompt)


'ek_temp_20200ek_temp_20084ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_200

In [17]:
import torch

gemma_model.model.embedder(torch.tensor([157]).to(gemma_model.device))[:,:15]

tensor([[-1.7912,  0.1602,  0.1088, -2.0236,  1.2162, -0.8047, -0.3950,  0.3219,
         -0.0877,  0.2307,  0.3475,  0.3288,  0.6641,  1.6956, -0.2081]],
       device='mps:0', grad_fn=<SliceBackward0>)

In [53]:
gemma_model.embedder.weight.shape

torch.Size([32768, 640])

In [16]:
model.model.embed_tokens(torch.tensor([30158]))[:,:15]

tensor([[ 0.4416,  0.9512,  1.0129, -0.0926, -0.5219, -1.2538, -0.1451, -1.1735,
          1.4885, -0.5281,  1.1859, -0.0598,  0.4107,  1.3094,  0.4045]],
       grad_fn=<SliceBackward0>)

In [18]:
model.model.layers[4].mlp.gate_proj.weight[234][:15], gemma_model.model.layers[4].mlp.gate_proj.weight[234][:15]

(tensor([ 0.0264, -0.0087,  0.0054,  0.0137, -0.0089,  0.0084,  0.0043, -0.0322,
          0.0408,  0.0034, -0.0100,  0.0056, -0.0010,  0.0203, -0.0325],
        grad_fn=<SliceBackward0>),
 tensor([ 0.0264, -0.0087,  0.0054,  0.0137, -0.0089,  0.0084,  0.0043, -0.0322,
          0.0408,  0.0034, -0.0100,  0.0056, -0.0010,  0.0203, -0.0325],
        device='mps:0', grad_fn=<SliceBackward0>))

In [14]:
gemma_model.save_pretrained("gemma-3-270m-tr-tokenizer-it")

Model saved to gemma-3-270m-tr-tokenizer-it


In [ ]:
import torch
tensors = tokenizer.encode(prompt)
tensors = torch.tensor(tensors)
tensors = tensors.unsqueeze(0)
tensors = tensors.to("cpu")

ids = model.generate(tensors)
ids

In [ ]:
tokenizer.decode(ids[0])

In [ ]:
model_1b = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-pt")
model_1b

In [ ]:
all_tr_tokens = tr_tokenizer.get_vocab()

print(len(all_tr_tokens))
print(len(tr_tokenizer.reverse_dict))
print(tr_tokenizer.vocab_size)

In [ ]:
all_tr_tokens["ali"]

In [12]:
from tokenizer_matcher import TokenizerMatcher


tokenizer_matcher = TokenizerMatcher(tokenizer, tr_tokenizer, model, gemma_model)
matched_embeddings = tokenizer_matcher.match_embeddings()

tokenizer_matcher.change_target_model_embeddings(matched_embeddings)

gemma_model.generate(prompt)

Changed 32763 embeddings out of 32763 matched tokens


'ek_temp_20200ek_temp_20084ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20074ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_20077ek_temp_200

In [36]:
matched_tokens = tokenizer_matcher.match_tokens()
matched_tokens[157]


[2, 30158]

In [37]:
tokenizer.vocab_size

262144

In [ ]:
token = "soğucuk"
ids = tokenizer.encode(token)
tr_ids = tr_tokenizer.encode(token)

print(ids)
print(tr_ids)

print(tokenizer.tokenize(token))
print(tr_tokenizer.tokenize(token))
print(tr_tokenizer.decode(tr_ids))

In [ ]:
example_ids = [157, 165, 5]


In [ ]:
tokenizer.eos_token_id

In [ ]:
model.model.state_dict().keys()


In [42]:
model.model.embed_tokens.weight[30158][:15]

tensor([ 0.0175,  0.0376,  0.0400, -0.0037, -0.0206, -0.0496, -0.0057, -0.0464,
         0.0588, -0.0209,  0.0469, -0.0024,  0.0162,  0.0518,  0.0160],
       grad_fn=<SliceBackward0>)

In [44]:
model.lm_head.weight[30158][:15]

tensor([ 0.0175,  0.0376,  0.0400, -0.0037, -0.0206, -0.0496, -0.0057, -0.0464,
         0.0588, -0.0209,  0.0469, -0.0024,  0.0162,  0.0518,  0.0160],
       grad_fn=<SliceBackward0>)